In [4]:
from platform import python_version

print(python_version())

3.8.11


To run the code, you will need to install the following things into an environment:

HuggingFace Transformers: pip install transformers.
A deep learning framework: either TensorFlow or PyTorch.
Torch Scatter, which is a TAPAS dependency. The command is dependent on whether you are using it with PyTorch GPU or CPU. Replace 1.6.0 with your PyTorch version.
For GPU: pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+${CUDA}.html
For CPU: pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+cpu.html

In [5]:
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+cpu.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cpu.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp38-cp38-win_amd64.whl size=319061 sha256=dec34ee1d2e312132aeb93212979b95934f98954f32eb7903b6ac82966b10cc5
  Stored in directory: c:\users\ramesh\appdata\local\pip\cache\wheels\7c\51\2a\409339f45a48bf748a5db76dfa11373ea7c883ecf1932eee2f
Successfully built torch-scatter
Note: you may need to restart the kernel to use updated packages.


Model code
Compared to Pipelines and other pretrained models, running TAPAS requires you to do a few more things. Below, you can find the code for the TAPAS based model as a whole. 

Imports: First of all, we’re importing the TapasTokenizer and TapasForQuestionAnswering imports from transformers – that is, HuggingFace Transformers. The tokenizer can be used for tokenization of which the result can be fed to the question answering model subsequently. Tapas requires a specific way of tokenization and input presenting, and these Tapas specific tokenizer and QA model have this built in. We also import pandas, which we’ll need later.
Table and question definitions: next up is defining the table and the questions. As you can see, the table is defined as a Python dictionary. Our table has two columns – Cities and Inhabitants – and values (in millions of inhabitants) are provided for Paris, London and Lyon.
Specifying some Python definitions:
Loading the model and tokenizer: in load_model_and_tokenizer, we initialize the Tokenizer and QuestionAnswering model with a finetuned variant of TAPAS – more specifically, google/tapas-base-finetuned-wtq, or TAPAS finetuned on WikiTable Questions (WTQ).
Preparing the inputs: our Python dictionary must first be converted into a DataFrame before it can be tokenized. We use pandas for this purpose, and create the dataframe from a dictionary. We can then feed it to the tokenizer together with the queries, and return the results.
Generating the predictions: in generate_predictions, we feed the tokenized inputs to our TAPAS model. Our tokenizer can be used subsequently to find the cell coordinates and aggregation operators that were predicted – recall that TAPAS predicts relevant cells (the coordinates) and an operator that must be executed to answer the question (the aggregation operator).
Postprocessing the predictions: in postprocess_predictions, we convert the predictions into a format that can be displayed on screen.
Showing the answers: in show_answers, we then actually visualize these answers.
Running TAPAS: run_tapas combines all other defs together in an end-to-end flow. This wasn’t directly added to __main__ because it’s best practice to keep as much functionality within Python definitions.
Running the whole thing: so far, we have created a lot of definitions, but nothing is running yet. That’s why we check whether our Python is running with that if statement at the bottom, and if so, invoke run_tapas() – and therefore the whole model.

In [1]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd

# Define the table
data = {'Cities': ["Paris, France", "London, England", "Lyon, France"], 'Inhabitants': ["2.161", "8.982", "0.513"]}

# Define the questions
queries = ["Which city has most inhabitants?", "What is the average number of inhabitants?", "How many French cities are in the list?", "How many inhabitants live in French cities?"]

def load_model_and_tokenizer():
  """
    Load
  """
  # Load pretrained tokenizer: TAPAS finetuned on WikiTable Questions
  tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

  # Load pretrained model: TAPAS finetuned on WikiTable Questions
  model = TapasForQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

  # Return tokenizer and model
  return tokenizer, model


def prepare_inputs(data, queries, tokenizer):
  """
    Convert dictionary into data frame and tokenize inputs given queries.
  """
  # Prepare inputs
  table = pd.DataFrame.from_dict(data)
  inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
  
  # Return things
  return table, inputs


def generate_predictions(inputs, model, tokenizer):
  """
    Generate predictions for some tokenized input.
  """
  # Generate model results
  outputs = model(**inputs)

  # Convert logit outputs into predictions for table cells and aggregation operators
  predicted_table_cell_coords, predicted_aggregation_operators = tokenizer.convert_logits_to_predictions(
          inputs,
          outputs.logits.detach(),
          outputs.logits_aggregation.detach()
  )
  
  # Return values
  return predicted_table_cell_coords, predicted_aggregation_operators


def postprocess_predictions(predicted_aggregation_operators, predicted_table_cell_coords, table):
  """
    Compute the predicted operation and nicely structure the answers.
  """
  # Process predicted aggregation operators
  aggregation_operators = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT"}
  aggregation_predictions_string = [aggregation_operators[x] for x in predicted_aggregation_operators]

  # Process predicted table cell coordinates
  answers = []
  for coordinates in predicted_table_cell_coords:
    if len(coordinates) == 1:
      # 1 cell
      answers.append(table.iat[coordinates[0]])
    else:
      # > 1 cell
      cell_values = []
      for coordinate in coordinates:
        cell_values.append(table.iat[coordinate])
      answers.append(", ".join(cell_values))
      
  # Return values
  return aggregation_predictions_string, answers


def show_answers(queries, answers, aggregation_predictions_string):
  """
    Visualize the postprocessed answers.
  """
  for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
    print(query)
    if predicted_agg == "NONE":
      print("Predicted answer: " + answer)
    else:
      print("Predicted answer: " + predicted_agg + " > " + answer)


def run_tapas():
  """
    Invoke the TAPAS model.
  """
  tokenizer, model = load_model_and_tokenizer()
  table, inputs = prepare_inputs(data, queries, tokenizer)
  predicted_table_cell_coords, predicted_aggregation_operators = generate_predictions(inputs, model, tokenizer)
  aggregation_predictions_string, answers = postprocess_predictions(predicted_aggregation_operators, predicted_table_cell_coords, table)
  show_answers(queries, answers, aggregation_predictions_string)


if __name__ == '__main__':
  run_tapas()

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Which city has most inhabitants?
Predicted answer: London, England
What is the average number of inhabitants?
Predicted answer: AVERAGE > 2.161, 8.982, 0.513
How many French cities are in the list?
Predicted answer: COUNT > Paris, France, Lyon, France
How many inhabitants live in French cities?
Predicted answer: SUM > 2.161, 0.513
